In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [21]:
# CPU / GPU 확인
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [22]:
EPOCHS = 30
BATCH_SIZE = 64

In [23]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True,
    transform = transform
)
testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform
)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,10)
        
    def forward(self, x):
        x = x.view(-1, 784) # 1차원 행렬로 변환
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [15]:
model = Net().to(DEVICE)

In [16]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [26]:
def train(model, train_loader, optimizer):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) # 학습 데이터를 DEVICE의 메모리로 보냄
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target) # 오차함수 : 클래스가 여러 개이므로 교차 엔트로피를 사용
        loss.backward()
        optimizer.step()

In [27]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad(): # 평가 단계에서는 기울기를 계산할 필요가 없으므로 no_grad()
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1] # 가장 큰 값 => prediction
            correct += pred.eq(target.view_as(pred)).sum().item() # 맞은 개수를 모두 합함
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [28]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.8286, Accuracy: 68.65%
[2] Test Loss: 0.6660, Accuracy: 75.35%
[3] Test Loss: 0.5697, Accuracy: 80.06%
[4] Test Loss: 0.5307, Accuracy: 81.43%
[5] Test Loss: 0.5099, Accuracy: 82.13%
[6] Test Loss: 0.4915, Accuracy: 82.59%
[7] Test Loss: 0.4918, Accuracy: 82.30%
[8] Test Loss: 0.4756, Accuracy: 83.22%
[9] Test Loss: 0.4705, Accuracy: 83.46%
[10] Test Loss: 0.4558, Accuracy: 83.64%
[11] Test Loss: 0.4432, Accuracy: 84.24%
[12] Test Loss: 0.4376, Accuracy: 84.43%
[13] Test Loss: 0.4438, Accuracy: 84.28%
[14] Test Loss: 0.4277, Accuracy: 84.82%
[15] Test Loss: 0.4252, Accuracy: 84.98%
[16] Test Loss: 0.4175, Accuracy: 85.26%
[17] Test Loss: 0.4147, Accuracy: 85.18%
[18] Test Loss: 0.4237, Accuracy: 84.90%
[19] Test Loss: 0.4053, Accuracy: 85.72%
[20] Test Loss: 0.3997, Accuracy: 85.87%
[21] Test Loss: 0.4017, Accuracy: 85.94%
[22] Test Loss: 0.3985, Accuracy: 85.60%
[23] Test Loss: 0.3838, Accuracy: 86.44%
[24] Test Loss: 0.3849, Accuracy: 86.29%
[25] Test Loss: 0.3837, A